#### atlas定義

In [ ]:
from nilearn import datasets, plotting, maskers
import numpy as np

atlas = datasets.fetch_atlas_aal(version='3v2')

atlas_filename = atlas.maps

coords = plotting.find_parcellation_cut_coords(atlas_filename)

print(f"Coordinates shape: {coords.shape}") # (166, 3)

## ABIDE AAL Connectome

#### 3D map

In [ ]:
import os

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import matplotlib.pyplot as plt
from nilearn import datasets, plotting
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import NiftiLabelsMasker

In [ ]:
# fetching a single subject's preprocessed functional data from the ABIDE dataset
dataset = datasets.fetch_abide_pcp(n_subjects=1, pipeline='cpac', band_pass_filtering=True)

# loading the AAL atlas for parcellation
atlas = datasets.fetch_atlas_aal()
labels = atlas.labels

# extracting time series from the functional data using the AAL atlas
masker = NiftiLabelsMasker(
    labels_img=atlas.maps, 
    standardize='zscore_sample', # type: ignore
    memory='nilearn_cache', 
    verbose=5
)
time_series = masker.fit_transform(dataset.func_preproc[0])

In [ ]:
correlation_measure = ConnectivityMeasure(
    kind='correlation', 
    standardize='zscore_sample'   # type: ignore
)
correlation_matrix = correlation_measure.fit_transform([time_series])[0]

In [ ]:
coords = plotting.find_parcellation_cut_coords(atlas.maps)
view = plotting.view_connectome(
    correlation_matrix, 
    coords[:correlation_matrix.shape[0]], 
    edge_threshold="98%"
)
view

#### 2D map

In [ ]:
from nilearn import plotting

plotting.plot_connectome(
    correlation_matrix, 
    coords[:correlation_matrix.shape[0]], 
    edge_threshold="99%",
    display_mode='ortho',
    node_size=20,
    edge_cmap='RdBu_r',
    title="ABIDE: Single Subject Connection Map (AAL)"
)

plotting.show()

#### Heatmap

In [ ]:
from nilearn import plotting

labels = atlas.labels[:correlation_matrix.shape[0]]

plotting.plot_matrix(
    correlation_matrix, 
    labels=labels, 
    colorbar=True, 
    vmax=1.0, 
    vmin=-1.0,
    title="Correlation Matrix (AAL Labels)"
)

#### Chord Diagram

In [ ]:
from mne_connectivity.viz import plot_connectivity_circle
import matplotlib.pyplot as plt

node_names = atlas.labels[:164]

fig, ax = plt.subplots(figsize=(8, 8), facecolor='black', subplot_kw=dict(polar=True))
plot_connectivity_circle(
    correlation_matrix, 
    node_names, 
    n_lines=20, 
    title='All-to-All Connectivity (AAL)',
    ax=ax
)
plt.show()

In [ ]:
import numpy as np
from mne_connectivity.viz import plot_connectivity_circle
import matplotlib.pyplot as plt

target_indices = [i for i, label in enumerate(atlas.labels[:164]) if 'Frontal' in label]
target_labels = [atlas.labels[i] for i in target_indices]

sub_matrix = correlation_matrix[np.ix_(target_indices, target_indices)]

fig, ax = plt.subplots(figsize=(10, 10), facecolor='black', subplot_kw=dict(polar=True))
plot_connectivity_circle(
    sub_matrix, 
    target_labels, 
    n_lines=50,
    title='Frontal Lobe Internal Connectivity',
    node_colors=['red' if '_L' in label else 'blue' for label in target_labels],
    ax=ax
)
plt.show()